In [1]:
import os

import dash
import dash_core_components as dcc
import dash_html_components as html

In [2]:
import pickle
import gzip

def load_pickle(fname):
  with gzip.open(fname, 'rb') as f:
     return pickle.load(f)

In [16]:
df = load_pickle("./data/df_20200404.pkl.gz")
references = load_pickle("./data/references_20200404.pkl.gz")
cpc = load_pickle("./data/cpc_20200404.pkl.gz")

In [17]:
df.tail(10)

,id,priority_date,assignee,abstract,claims,description
851,CN105846885A,2016-03-21,南京邮电大学,Abstract The invention provides a GEO satellit...,Claims ( 5 ) 1. a GEO satellite channel assign...,Description GEO satellite channel assignment s...
852,CN106253964A,2015-06-09,北京信威通信技术股份有限公司,Abstract The present invention proposes a kind...,Claims ( 22 ) 1. a mobile communication system...,Description Mobile communication system based ...
853,CN106357321A,2016-09-08,湖北航天技术研究院总体设计所,Abstract The invention discloses a space-based...,Claims ( 3 ) 1. a kind of space-based Internet...,Description A kind of space-based Internet of ...
854,CN106792946A,2017-01-12,北京邮电大学,Abstract The invention discloses a kind of ada...,Claims ( 5 ) 1. a kind of adaptive method for ...,Description A kind of adaptive method for swit...
855,CN106850045A,2017-01-17,北京邮电大学,Abstract LEO satellite system channel distribu...,Claims ( 4 ) 1. a kind of self adaptation over...,Description A kind of self adaptation overbook...
856,CN107241134A,2017-06-28,东北大学,Abstract The present invention relates to a ki...,Claims ( 10 ) 1. a kind of method for channel ...,Description A kind of method for channel alloc...
857,CN107800472A,2017-11-23,中国空间技术研究院,Abstract The method for handover control and s...,Claims ( 10 ) 1. a kind of satellite network m...,Description A kind of method for handover cont...
858,CN109474326A,2018-11-16,清华大学,Abstract The present invention provides a kind...,Claims ( 10 ) 1. a kind of beam switching meth...,Description Beam switching method and device T...
859,WO2004073229A2,2003-01-28,The Boeing Company,"Abstract A digital payload [202, 300, 500, 600...",Claims CLAIMS What is claimed is: 1. A digital...,Description SYSTEMS AND METHODS FOR DIGITAL PR...
860,CN101106417A,2007-08-20,北京航空航天大学,Abstract An on-board call access control metho...,Claims ( 5 ) 1. satellite call access control ...,Description A kind of satellite call access co...


In [18]:
import string
import re
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

def preprocess(text):
  text = text.lower()
  text = text.translate(str.maketrans('','',string.punctuation))
  tmp = [w for w in text.split() if w not in stop_words]
  text = " ".join([w for w in tmp if not re.match(r"^[0-9]{1,5}[a-z]$|^[0-9]{1,5}.*[0-9]$|^\(.*\)$|\\n|\\t|^\\", w)])
  return text

[nltk_data] Downloading package stopwords to /home/susumu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
df = df.drop_duplicates(subset="id")

In [20]:
import pandas as pd

preamble = "A health monitoring system for a plurality of satellites. "
text_A = "The system receives telemetry data from the plurality of satellites. The data is measured by the sensor which mounted on the satellite and received by the system on the ground. "
text_B = "The data is in a format specific to each satellite. The system converts the data to a standard format. The standard format of the data is independent of the hardware configuration of the satellite. "
text_C = "The system displays the data in the standard format. The system displays the data in a visualization format that is intuitive to understand for a human operator. The system displays the data via a web browser. "
text_D = "The health of the plurality of satellites is judged baesd on machine larning algorithms . When fault is detected, an alarm will be noticed to the operator. "

dic = {k:v for k,v in zip(df.columns, df.iloc[-1].values)}
dic["id"] = ["Candidate claim", "Without A", "Without B", "Without C", "Without D"]
dic["priority_date"] = ["2020-04-01"]*5
dic["assignee"] = ["Client"]*5
dic["abstract"] = [preamble+text_A+text_B+text_C+text_D, preamble+text_B+text_C+text_D, preamble+text_A+text_C+text_D, preamble+text_A+text_B+text_D, preamble+text_A+text_B+text_C]
dic["claims"] = [preamble+text_A+text_B+text_C+text_D, preamble+text_B+text_C+text_D, preamble+text_A+text_C+text_D, preamble+text_A+text_B+text_D, preamble+text_A+text_B+text_C]
dic["description"] = [preamble+text_A+text_B+text_C+text_D, preamble+text_B+text_C+text_D, preamble+text_A+text_C+text_D, preamble+text_A+text_B+text_D, preamble+text_A+text_B+text_C]
df = pd.concat([df,pd.DataFrame(dic)],ignore_index=True)

In [21]:
df["abstract_preprocess"] = df["abstract"].map(preprocess)
df["claims_preprocess"] = df["claims"].map(preprocess)
df["description_preprocess"] = df["description"].map(preprocess)
df.tail(10)

,id,priority_date,assignee,abstract,claims,description,abstract_preprocess,claims_preprocess,description_preprocess
732,CN107241134A,2017-06-28,东北大学,Abstract The present invention relates to a ki...,Claims ( 10 ) 1. a kind of method for channel ...,Description A kind of method for channel alloc...,abstract present invention relates kind method...,claims 1 kind method channel allocation leo sa...,description kind method channel allocation leo...
733,CN107800472A,2017-11-23,中国空间技术研究院,Abstract The method for handover control and s...,Claims ( 10 ) 1. a kind of satellite network m...,Description A kind of method for handover cont...,abstract method handover control system based ...,claims 1 kind satellite network method handove...,description kind method handover control syste...
734,CN109474326A,2018-11-16,清华大学,Abstract The present invention provides a kind...,Claims ( 10 ) 1. a kind of beam switching meth...,Description Beam switching method and device T...,abstract present invention provides kind beam ...,claims 1 kind beam switching method characteri...,description beam switching method device techn...
735,WO2004073229A2,2003-01-28,The Boeing Company,"Abstract A digital payload [202, 300, 500, 600...",Claims CLAIMS What is claimed is: 1. A digital...,Description SYSTEMS AND METHODS FOR DIGITAL PR...,abstract digital payload processing subband sp...,claims claims claimed 1 digital payload proces...,description systems methods digital processing...
736,CN101106417A,2007-08-20,北京航空航天大学,Abstract An on-board call access control metho...,Claims ( 5 ) 1. satellite call access control ...,Description A kind of satellite call access co...,abstract onboard call access control method ch...,claims 5 1 satellite call access control metho...,description kind satellite call access control...
737,Candidate claim,2020-04-01,Client,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...
738,Without A,2020-04-01,Client,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...
739,Without B,2020-04-01,Client,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...
740,Without C,2020-04-01,Client,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...
741,Without D,2020-04-01,Client,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,A health monitoring system for a plurality of ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...,health monitoring system plurality satellites ...


In [12]:
# from gensim.models.fasttext import load_facebook_model

# model = load_facebook_model('./crawl-300d-2M-subword.bin')

import gensim.downloader as api
model = api.load('word2vec-google-news-300')

KeyboardInterrupt: 

In [13]:
model.most_similar("satellite")

[('satellites', 0.767896831035614),
 ('Satellite', 0.7367128133773804),
 ('geostationary_communications', 0.6455782651901245),
 ('geostationary_satellite', 0.6399903297424316),
 ('sattelite', 0.6370919346809387),
 ('NASA_GeneSat_1', 0.6245810985565186),
 ('Gadaf_spoke', 0.6196396946907043),
 ('iPSTAR_broadband', 0.618923008441925),
 ('geo_stationary_satellite', 0.6126393675804138),
 ('geostationary_satellites', 0.6115455031394958)]

In [14]:
model.save("./word2vec-google-news-300.model")

In [11]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load("./word2vec-google-news-300.model")

AttributeError: Can't get attribute 'Word2VecKeyedVectors' on <module 'gensim.models.deprecated.keyedvectors' from '/home/susumu/.local/share/virtualenvs/dash_app_test-XxEw_oSZ/lib/python3.6/site-packages/gensim/models/deprecated/keyedvectors.py'>

In [15]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|                       In|       192|
|                      Out|       240|
|                      api|        80|
|                      cpc|     36976|
|                     dash|        80|
|                      dcc|        80|
|                       df|  29316118|
|                      dic|       368|
|                 download|        64|
|                     exit|        56|
|              get_ipython|        64|
|                     gzip|        80|
|                     html|        80|
|      load_facebook_model|       136|
|    load_facebook_vectors|       136|
|              load_pickle|       136|
|                    model|        56|
|                       os|        80|
|                       pd|        80|
|                   pickle|        80|
|                 preamble|       107|
|               preprocess|       136|
|                     quit|        56|
|                       r

In [28]:
def calc_WMD(text, id_texts):
    result = []
    for num, a, t in id_texts:
        if a == "abstract":
            result.append( (num, model.wmdistance(text.split(), t.split())) )
        else:
            result.append( (num, model.wmdistance(text.split(), a.split())) )
    return result  

In [29]:
%%time
result = []
for target in df["abstract"].values[-5:]:
    tmp = calc_WMD(target, df[["id","abstract","claims"]].values[:-5])
    sorted_tmp = sorted(tmp, key=lambda x: x[1])
    result.append([t[0] for t in sorted_tmp])
    print("{} done".format(len(result)))

1 done
2 done
3 done
4 done
5 done
CPU times: user 2min 24s, sys: 406 ms, total: 2min 25s
Wall time: 2min 26s


In [30]:
sorted_result = result[0]
sorted_result

['KR100554508B1',
 'KR101278554B1',
 'US8135338B1',
 'US9944414B2',
 'US5313215A',
 'CN102904651A',
 'US6549162B1',
 'US8498760B2',
 'US20150301190A1',
 'US8483888B2',
 'JP2000131474A',
 'US6683563B2',
 'US5864747A',
 'CN106528281A',
 'CN104618171A',
 'CN103281113A',
 'KR100758275B1',
 'EP2354803A1',
 'CN102323941A',
 'CN100361440C',
 'KR20140136706A',
 'US20040001479A1',
 'US5926745A',
 'KR100580087B1',
 'CN105574178A',
 'CN104597854A',
 'CN105549382A',
 'US5187805A',
 'CN104618311A',
 'US5119504A',
 'US20050090265A1',
 'CN103412963A',
 'KR20090038734A',
 'CN107046438A',
 'CN101535832A',
 'US6211819B1',
 'CN105574178B',
 'CN102542002A',
 'KR19990042177A',
 'US6173176B1',
 'KR20040055552A',
 'US20030204371A1',
 'US6185409B1',
 'JP2005229448A',
 'CN1898957B',
 'US6463366B2',
 'SE527599C2',
 'CN102968401A',
 'US20090093959A1',
 'JP2015195531A',
 'CN104408050A',
 'US20180013484A1',
 'JP6298176B2',
 'US20030176969A1',
 'KR100794375B1',
 'US6724737B1',
 'US20020065698A1',
 'CN104331035B',
 

In [35]:
import pickle
import gzip

with gzip.open("./data/results.pkl.gz", mode='wb') as f:
    f.write(pickle.dumps(result))

In [22]:
from collections import defaultdict

TOP=10

node_dict=defaultdict(int)
edge_list=[]
i = 1

for num in df["id"][:-5]:
  if not node_dict[num]:
    node_dict[num] = i
    dic = references[num]
    i += 1

  num_list = [url.split(sep="/")[-2] for url in dic["ForwardReferences"]]
  for n in num_list:
    if not node_dict[n]:
      if n in df["id"].values:
        node_dict[n] = i
        i += 1
        edge_list.append( (node_dict[num], node_dict[n]) )

  # num_list = [url.split(sep="/")[-2] for url in dic["BackwardReferences"]]
  # for n in num_list:
  #   if not node_dict[n]:
  #     if n in df["id"].values:
  #       node_dict[n] = i
  #       i += 1
  #       edge_list.append( (node_dict[n], node_dict[num]) )

i += 1
target = i
j = 0
for m in range(50):
  n = node_dict[sorted_result[m]]
  if n > 0:
    edge_list.append( (i, n) )
    j += 1
  if j >= TOP:
    break

In [23]:
num_to_id_dict = {v:k for k,v in node_dict.items()}
len(num_to_id_dict)

738

In [15]:
edge_list[-12:]

[(622, 624),
 (627, 628),
 (636, 637),
 (642, 643),
 (644, 645),
 (644, 646),
 (677, 678),
 (677, 679),
 (680, 681),
 (682, 683),
 (691, 692),
 (714, 715)]

In [41]:
import plotly.graph_objects as go
import networkx as nx

G = nx.Graph()

In [42]:
%%time
import math

G.add_edges_from(edge_list)
pos = nx.spring_layout(G, k=1.2/math.log2(len(edge_list)), pos={target:(0,0)} )

# pos={target:(0,0)}, fixed=[target]

CPU times: user 547 ms, sys: 62.5 ms, total: 609 ms
Wall time: 650 ms


In [43]:
for node in G.nodes:
  G.nodes[node]['pos'] = pos[node]

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Portland',
        reversescale=True,
        color=[],
        size=[],
        colorbar=dict(
            thickness=15,
            title='Forward Citation Count',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [44]:
forward_citation_count = []
node_text = []
for node in G.nodes():
  if node == target:
    forward_citation_count.append(10)
    node_text.append("Candidate claim")
    continue
  forward_citation_count.append(len(references[num_to_id_dict[node]]["ForwardReferences"] ))
  text = "<a href='https://patents.google.com/patent/{}/en'>{}</a>".format(num_to_id_dict[node], num_to_id_dict[node]) +", " + "{}".format(df[df["id"]==num_to_id_dict[node]]["assignee"].values[0]) 
  node_text.append(text)

node_trace.marker.color = forward_citation_count
node_trace.marker.size = [min(5 + (elm)**0.6, 30) for elm in forward_citation_count]
node_trace.text = node_text

In [46]:
import math

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Patent citation network',
                titlefont_size=20,
                showlegend=True,
                hovermode='closest',
                # margin=dict(b=20,l=5,r=5,t=40),
                # xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                # yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                width=1200, height=1000)
                )

annotations = []
X, Y = G.nodes[target]['pos']
theta = math.atan2(Y, X)
r = max([(X**2 + Y**2)**0.5, 0.05])
d = dict(
    x=X,
    y=Y,
    text="Candidate Claim",
    showarrow=True,
    arrowhead=7,
    ax=10*math.cos(theta) * r**-1,
    ay=-10*math.sin(theta) * r**-1)
annotations.append(d)

fig.update_layout(
    showlegend=False,
    annotations=annotations)

fig.show()